In [1]:
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data

c:\users\jordy\anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
tf.set_random_seed(777)  # reproducibility
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

# Parâmetros de aprendizagem

In [10]:
taxa_aprendizado = 0.001
quantidade_maxima_epocas = 15
batch_size = 100

# Rede Convolucional

**Entrada dos place holders**

In [5]:
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])   # imagem 28x28x1 (preto e branca)
Y = tf.placeholder(tf.float32, [None, 10])

**L1 ImgIn shape=(?, 28, 28, 1)**
* Conv -> (?, 28, 28, 32)
* Pool -> (?, 14, 14, 32)

In [14]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X_img, W1, 
                  strides=[1, 1, 1, 1], 
                  padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], 
                    padding='SAME')

# Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
# Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
# Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)

**L2 ImgIn shape=(?, 14, 14, 32)**
* Conv -> (?, 14, 14, 64)
* Pool -> (?, 7, 7, 64)

In [13]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], 
                      padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], 
                    padding='SAME')
L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])

# Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
# Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
# Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
# Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)

**Classificador - Camada Fully Connected entrada 7x7x64 -> 10 saídas**

In [8]:
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b

**Define a função de custo e o método de otimização**

In [12]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=taxa_aprendizado).minimize(cost)

# Inicializa


In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Treinamento

In [16]:
print('Rede inicialiada. Treinamento inicializado. Tome um cafe...')
for epoca in range(quantidade_maxima_epocas):
    custo_medio = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        custo_medio += c / total_batch

    print('Epoca:', '%04d' % (epoca + 1), 'perda =', '{:.9f}'.format(custo_medio))

print('Treinamento finalizado!')

Rede inicialiada. Treinamento inicializado. Tome um cafe...
Epoca: 0001 perda = 0.345651393
Epoca: 0002 perda = 0.091837431
Epoca: 0003 perda = 0.068186304
Epoca: 0004 perda = 0.056457348
Epoca: 0005 perda = 0.046903238
Epoca: 0006 perda = 0.041060929
Epoca: 0007 perda = 0.036540111
Epoca: 0008 perda = 0.032742291
Epoca: 0009 perda = 0.027917723
Epoca: 0010 perda = 0.025122694
Epoca: 0011 perda = 0.022135002
Epoca: 0012 perda = 0.020564338
Epoca: 0013 perda = 0.017400479
Epoca: 0014 perda = 0.015662067
Epoca: 0015 perda = 0.013344748
Treinamento finalizado!


# Teste o modelo e verifica a taxa de acerto

In [27]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
score =  sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}) * 100
print("Taxa de acerto: % .3f%%" % score)

Taxa de acerto:  98.870%


**Obtém uma nova imagem e testa o modelo**

In [29]:
r = random.randint(0, mnist.test.num_examples - 1)
print("Classe real: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Predição:    ", sess.run(
    tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

Classe real:  [5]
Predição:     [5]
